In [ ]:
# Install Unsloth & Dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os
import json

In [ ]:
# Load Configuration from Env (Simulated via file/custom logic)
# In Kaggle, we can pass config via dataset or hardcoded for now.
MODEL_NAME = "unsloth/llama-3-8b-bn"
MAX_SEQ_LENGTH = 2048
DTYPE = None # float16 for Tesla T4, bfloat16 for Ampere+
LOAD_IN_4BIT = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
# Load Dataset from Kaggle Input
# Assuming dataset name was passed or standard path
# /kaggle/input/{user}/{slug}/{filename}
dataset_path = "/kaggle/input/dataset.json" # Placeholder
data_files = {"train": dataset_path}
dataset = load_dataset("json", data_files=data_files, split="train")

def formatting_prompts_func(examples):
    texts = examples["text"]
    labels = examples["label"]
    outputs = []
    for text, label in zip(texts, labels):
        text = f"Sentiment Analysis:\nInput: {text}\nOutput: {label}"
        outputs.append(text)
    return { "text" : outputs, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Small step count for faster testing
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# Export LoRA adapters
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")